# 第八章 錯誤

在 Go 語言中，錯誤處理是一個核心概念。與其他語言使用例外處理不同，Go 使用顯式的錯誤值來處理錯誤情況。這種設計哲學讓錯誤處理變得明確和可預測，開發者必須主動處理每一個可能的錯誤。

Go 的錯誤處理基於一個簡單的介面：`error`，它只有一個 `Error()` 方法。這種簡單的設計讓錯誤處理既靈活又一致。

## 如何處理錯誤：基本知識

Go 中的錯誤處理遵循一個簡單的模式：函式返回一個錯誤值，呼叫者檢查這個值來決定如何處理錯誤。這種明確的錯誤處理方式讓程式的控制流程更加清晰。

基本的錯誤處理模式如下：
1. 函式返回 `(result, error)` 的組合
2. 呼叫者檢查 error 是否為 nil
3. 如果 error 不為 nil，處理錯誤情況
4. 如果 error 為 nil，使用返回的結果

In [1]:
package main

import (
    "fmt"
    "strconv"
)

func main() {
    /* 基本錯誤處理模式 - 成功案例 */
    numStr := "123"
    num, err := strconv.Atoi(numStr)
    if err != nil {
        fmt.Println("轉換失敗:", err)
        return
    }
    fmt.Println("轉換成功:", num) // 輸出: 轉換成功: 123
}

轉換成功: 123


In [2]:
package main

import (
    "fmt"
    "strconv"
)

func main() {
    /* 基本錯誤處理模式 - 錯誤案例 */
    invalidStr := "abc"
    num, err := strconv.Atoi(invalidStr)
    if err != nil {
        fmt.Println("轉換失敗:", err) // 輸出: 轉換失敗: strconv.Atoi: parsing "abc": invalid syntax
        return
    }
    fmt.Println("轉換成功:", num) // 這行不會執行
}

轉換失敗: strconv.Atoi: parsing "abc": invalid syntax


In [ ]:
package main

import (
    "fmt"
    "os"
)

func main() {
    /* 文件操作的錯誤處理 */
    file, err := os.Open("nonexistent.txt")
    if err != nil {
        fmt.Println("開啟文件失敗:", err) // 輸出: 開啟文件失敗: open nonexistent.txt: no such file or directory
        return
    }
    defer file.Close() // 如果成功開啟，記得關閉文件
    fmt.Println("文件開啟成功")
}

## 用字串來類示簡單的錯誤

最簡單的錯誤建立方式是使用 `errors.New()` 函式或 `fmt.Errorf()` 函式。這些函式建立包含錯誤訊息的簡單錯誤值。

- `errors.New(string)`: 建立包含固定訊息的錯誤
- `fmt.Errorf(format, args...)`: 建立包含格式化訊息的錯誤

這種方式適合簡單的錯誤情況，當你只需要提供錯誤描述而不需要額外的錯誤資訊時。

In [ ]:
package main

import (
    "errors"
    "fmt"
)

func divide(a, b float64) (float64, error) {
    /* 使用 errors.New 建立簡單錯誤 */
    if b == 0 {
        return 0, errors.New("除數不能為零")
    }
    return a / b, nil
}

func main() {
    result, err := divide(10, 0)
    if err != nil {
        fmt.Println("錯誤:", err) // 輸出: 錯誤: 除數不能為零
        return
    }
    fmt.Println("結果:", result)
}

In [3]:
package main

import "fmt"

func processAge(age int) error {
    /* 使用 fmt.Errorf 建立格式化錯誤訊息 */
    if age < 0 {
        return fmt.Errorf("年齡不能為負數: %d", age)
    }
    if age > 150 {
        return fmt.Errorf("年齡過大，不合理: %d", age)
    }
    fmt.Println("年齡有效:", age)
    return nil
}

func main() {
    ages := []int{25, -5, 200}
    for _, age := range ages {
        if err := processAge(age); err != nil {
            fmt.Println("處理年齡錯誤:", err)
        }
    }
    // 輸出:
    // 年齡有效: 25
    // 處理年齡錯誤: 年齡不能為負數: -5
    // 處理年齡錯誤: 年齡過大，不合理: 200
}

年齡有效: 25
處理年齡錯誤: 年齡不能為負數: -5
處理年齡錯誤: 年齡過大，不合理: 200


## 哨兵錯誤 Sentinel Error

哨兵錯誤是預先定義的錯誤值，用於表示特定的錯誤條件。它們通常定義為套件層級的變數，以 `Err` 開頭命名。哨兵錯誤讓呼叫者能夠識別特定類型的錯誤並採取相應的行動。

使用哨兵錯誤的優點：
- 可以使用 `==` 來比較特定錯誤
- 提供了一致的錯誤識別方式
- 讓錯誤處理邏輯更加明確

常見的哨兵錯誤例如：`io.EOF`、`os.ErrNotExist` 等。

**注意** : 你一旦使用哨兵錯誤, 就代表他是公用API的一部分了, 代表你已經承諾將來所有回朔相容版本他都可以使用; 更好的作法是重複使用標準程式庫裡面已經有定義的error 

In [ ]:
package main

import (
    "errors"
    "fmt"
)

/* 定義哨兵錯誤 */
var (
    ErrNotFound    = errors.New("項目未找到")
    ErrInvalidType = errors.New("無效的類型")
    ErrOutOfRange  = errors.New("索引超出範圍")
)

func getItem(items []string, index int) (string, error) {
    /* 使用哨兵錯誤返回特定錯誤條件 */
    if len(items) == 0 {
        return "", ErrNotFound
    }
    if index < 0 || index >= len(items) {
        return "", ErrOutOfRange
    }
    return items[index], nil
}

func main() {
    items := []string{"apple", "banana", "cherry"}
    
    item, err := getItem(items, 5)
    if err == ErrOutOfRange {
        fmt.Println("索引超出範圍錯誤") // 輸出: 索引超出範圍錯誤
    } else if err == ErrNotFound {
        fmt.Println("項目未找到錯誤")
    } else if err != nil {
        fmt.Println("其他錯誤:", err)
    } else {
        fmt.Println("項目:", item)
    }
}

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

func main() {
    /* 使用標準函式庫的哨兵錯誤 io.EOF */
    reader := strings.NewReader("Hello")
    
    for {
        var b [1]byte
        n, err := reader.Read(b[:])
        
        if err == io.EOF {
            fmt.Println("讀取完成") // 當讀完所有資料時輸出
            break
        }
        if err != nil {
            fmt.Println("讀取錯誤:", err)
            break
        }
        if n > 0 {
            fmt.Printf("讀取到: %c\n", b[0]) // 輸出: H, e, l, l, o
        }
    }
}

## error 是值

在 Go 中，error 是一個介面類型，可以像其他值一樣被賦值、傳遞和操作。你可以建立自訂的錯誤類型來包含更豐富的錯誤資訊，只要實作 `Error() string` 方法即可。

自訂錯誤類型的優勢：
- 可以包含額外的上下文資訊
- 可以提供結構化的錯誤資料
- 可以實作特定的錯誤行為
- 允許類型斷言來獲取詳細資訊

In [ ]:
package main

import "fmt"

/* 定義自訂錯誤類型 */
type ValidationError struct {
    Field   string
    Value   interface{}
    Message string
}

/* 實作 Error() 方法 */
func (ve ValidationError) Error() string {
    return fmt.Sprintf("驗證錯誤 - 欄位: %s, 值: %v, 訊息: %s", ve.Field, ve.Value, ve.Message)
}

func validateAge(age int) error {
    /* 返回自訂錯誤類型 */
    if age < 0 {
        return ValidationError{
            Field:   "age",
            Value:   age,
            Message: "年齡不能為負數",
        }
    }
    return nil
}

func main() {
    err := validateAge(-5)
    if err != nil {
        fmt.Println(err) // 輸出: 驗證錯誤 - 欄位: age, 值: -5, 訊息: 年齡不能為負數
    }
}

In [ ]:
package main

import "fmt"

/* 包含錯誤碼的自訂錯誤 */
type APIError struct {
    StatusCode int
    Message    string
    Details    map[string]interface{}
}

func (ae APIError) Error() string {
    return fmt.Sprintf("API 錯誤 [%d]: %s", ae.StatusCode, ae.Message)
}

/* 提供額外方法來獲取錯誤資訊 */
func (ae APIError) IsClientError() bool {
    return ae.StatusCode >= 400 && ae.StatusCode < 500
}

func (ae APIError) IsServerError() bool {
    return ae.StatusCode >= 500
}

func callAPI() error {
    /* 模擬 API 呼叫返回錯誤 */
    return APIError{
        StatusCode: 404,
        Message:    "資源未找到",
        Details:    map[string]interface{}{"resource": "user", "id": 123},
    }
}

func main() {
    err := callAPI()
    if err != nil {
        fmt.Println("錯誤:", err) // 輸出: 錯誤: API 錯誤 [404]: 資源未找到
        
        /* 使用類型斷言獲取詳細資訊 */
        if apiErr, ok := err.(APIError); ok {
            fmt.Println("狀態碼:", apiErr.StatusCode)              // 輸出: 狀態碼: 404
            fmt.Println("是客戶端錯誤:", apiErr.IsClientError())    // 輸出: 是客戶端錯誤: true
            fmt.Println("詳細資訊:", apiErr.Details)               // 輸出: 詳細資訊: map[id:123 resource:user]
        }
    }
}

## 包裝錯誤

錯誤包裝是 Go 1.13 引入的功能，允許你在保留原始錯誤的同時添加額外的上下文資訊。使用 `fmt.Errorf` 配合 `%w` 動詞可以包裝錯誤，這樣可以建立錯誤鏈(Error Chain)，讓錯誤追蹤更加完整。

包裝錯誤的好處：
- 保留原始錯誤資訊
- 添加呼叫層級的上下文
- 支援錯誤解包裝和檢查
- 提供更豐富的錯誤追蹤資訊

### 可以使用 errors.Unwrap 來解開錯誤鏈 

使用 `errors.Unwrap(err)` 可以解開來看是不是有包其他的error, 
- `有內層error` : 回傳該error
- `沒有內層error` : 回傳nil

```go
if unwrapped := errors.Unwrap(err); unwrapped != nil {
        fmt.Println("內層錯誤:", unwrapped)
}
```
        

In [4]:
package main

import (
    "errors"
    "fmt"
    "strconv"
)

func parseConfig(configStr string) (int, error) {
    /* 基層函式，產生原始錯誤 */
    value, err := strconv.Atoi(configStr)
    if err != nil {
        // 使用 %w 包裝錯誤，添加上下文資訊
        return 0, fmt.Errorf("解析配置值失敗: %w", err)
    }
    return value, nil
}

func loadConfiguration() error {
    /* 中層函式，再次包裝錯誤 */
    _, err := parseConfig("invalid")
    if err != nil {
        return fmt.Errorf("載入配置失敗: %w", err)
    }
    return nil
}

func main() {
    /* 頂層函式，處理包裝後的錯誤 */
    err := loadConfiguration()
    if err != nil {
        fmt.Println("完整錯誤:", err)
        // 輸出: 完整錯誤: 載入配置失敗: 解析配置值失敗: strconv.Atoi: parsing "invalid": invalid syntax
    }
}

完整錯誤: 載入配置失敗: 解析配置值失敗: strconv.Atoi: parsing "invalid": invalid syntax


In [ ]:
package main

import (
    "errors"
    "fmt"
    "os"
)

func readUserConfig(filename string) error {
    /* 讀取檔案並包裝可能的錯誤 */
    _, err := os.ReadFile(filename)
    if err != nil {
        return fmt.Errorf("無法讀取使用者配置檔案 '%s': %w", filename, err)
    }
    return nil
}

func initializeApp() error {
    /* 初始化應用程式，可能呼叫多個可能失敗的操作 */
    err := readUserConfig("nonexistent.conf")
    if err != nil {
        return fmt.Errorf("應用程式初始化失敗: %w", err)
    }
    return nil
}

func main() {
    err := initializeApp()
    if err != nil {
        fmt.Println("錯誤:", err)
        // 輸出類似: 錯誤: 應用程式初始化失敗: 無法讀取使用者配置檔案 'nonexistent.conf': open nonexistent.conf: no such file or directory
        
        /* 可以使用 errors.Unwrap 來解開錯誤鏈 */
        if unwrapped := errors.Unwrap(err); unwrapped != nil {
            fmt.Println("內層錯誤:", unwrapped)
        }
    }
}

## Is 與 As

Go 1.13 引入了 `errors.Is` 和 `errors.As` 函式，用於檢查包裝錯誤鏈中的特定錯誤。這些函式可以透過錯誤鏈來查找特定的錯誤類型或錯誤值。

- `errors.Is(err, target)`: 檢查錯誤鏈中是否包含特定的錯誤值
- `errors.As(err, target)`: 檢查錯誤鏈中是否包含特定類型的錯誤，並提取該錯誤

這些函式替代了直接的錯誤比較，在處理包裝錯誤時更加可靠。

In [ ]:
package main

import (
    "errors"
    "fmt"
    "os"
)

func processFile(filename string) error {
    /* 嘗試開啟檔案，包裝可能的錯誤 */
    _, err := os.Open(filename)
    if err != nil {
        return fmt.Errorf("處理檔案 '%s' 失敗: %w", filename, err)
    }
    return nil
}

func main() {
    err := processFile("nonexistent.txt")
    if err != nil {
        /* 使用 errors.Is 檢查包裝錯誤鏈中的特定錯誤 */
        if errors.Is(err, os.ErrNotExist) {
            fmt.Println("檔案不存在錯誤") // 輸出: 檔案不存在錯誤
        } else {
            fmt.Println("其他錯誤:", err)
        }
        
        /* 直接比較會失敗，因為錯誤被包裝了 */
        if err == os.ErrNotExist {
            fmt.Println("這行不會執行，因為錯誤被包裝了")
        } else {
            fmt.Println("直接比較失敗，因為是包裝錯誤") // 這行會執行
        }
    }
}

In [ ]:
package main

import (
    "errors"
    "fmt"
    "net"
)

func connectToServer() error {
    /* 模擬網路連接錯誤 */
    _, err := net.Dial("tcp", "nonexistent:8080")
    if err != nil {
        return fmt.Errorf("連接伺服器失敗: %w", err)
    }
    return nil
}

func main() {
    err := connectToServer()
    if err != nil {
        /* 使用 errors.As 提取特定類型的錯誤 */
        var netErr *net.OpError
        if errors.As(err, &netErr) {
            fmt.Println("網路操作錯誤:", netErr.Op) // 輸出網路操作類型
            fmt.Println("網路地址:", netErr.Net)    // 輸出網路類型
            fmt.Println("錯誤地址:", netErr.Addr)   // 輸出錯誤地址
        }
        
        /* 也可以檢查其他類型的錯誤 */
        var dnsErr *net.DNSError
        if errors.As(err, &dnsErr) {
            fmt.Println("DNS 錯誤:", dnsErr.Name)
            fmt.Println("是否為暫時性錯誤:", dnsErr.Temporary())
        }
    }
}

In [ ]:
package main

import (
    "errors"
    "fmt"
)

/* 自訂錯誤類型 */
type TimeoutError struct {
    Operation string
    Duration  int
}

func (te TimeoutError) Error() string {
    return fmt.Sprintf("操作 '%s' 超時 (%d 秒)", te.Operation, te.Duration)
}

func performOperation() error {
    /* 模擬超時錯誤並包裝 */
    timeoutErr := TimeoutError{Operation: "database_query", Duration: 30}
    return fmt.Errorf("操作執行失敗: %w", timeoutErr)
}

func main() {
    err := performOperation()
    if err != nil {
        /* 使用 errors.As 提取自訂錯誤類型 */
        var timeoutErr TimeoutError
        if errors.As(err, &timeoutErr) {
            fmt.Println("發生超時錯誤")                          // 輸出: 發生超時錯誤
            fmt.Println("操作:", timeoutErr.Operation)        // 輸出: 操作: database_query
            fmt.Println("超時時間:", timeoutErr.Duration, "秒") // 輸出: 超時時間: 30 秒
        }
    }
}

## 用 defer 來包裝 error

`defer` 語句可以與錯誤處理結合使用，特別是在需要在函式返回前修改錯誤或添加額外上下文時。這種技術通常用於：

- 在函式返回前統一處理錯誤
- 為錯誤添加函式級別的上下文
- 實現錯誤恢復機制
- 記錄錯誤日誌

使用 defer 包裝錯誤需要使用具名返回值，這樣才能在 defer 函式中修改返回的錯誤。

In [ ]:
package main

import (
    "fmt"
    "strconv"
)

func parseAndValidate(s string) (result int, err error) {
    /* 使用具名返回值，方便在 defer 中修改 */
    defer func() {
        if err != nil {
            // 在 defer 中包裝錯誤，添加函式級別的上下文
            err = fmt.Errorf("parseAndValidate('%s') 失敗: %w", s, err)
        }
    }()
    
    /* 解析字串為整數 */
    result, err = strconv.Atoi(s)
    if err != nil {
        return 0, err // 這個錯誤會被 defer 包裝
    }
    
    /* 驗證範圍 */
    if result < 0 || result > 100 {
        return 0, fmt.Errorf("值 %d 超出有效範圍 [0, 100]", result)
    }
    
    return result, nil
}

func main() {
    testCases := []string{"abc", "150", "50"}
    
    for _, tc := range testCases {
        result, err := parseAndValidate(tc)
        if err != nil {
            fmt.Println("錯誤:", err)
        } else {
            fmt.Println("成功:", result)
        }
    }
    // 輸出:
    // 錯誤: parseAndValidate('abc') 失敗: strconv.Atoi: parsing "abc": invalid syntax
    // 錯誤: parseAndValidate('150') 失敗: 值 150 超出有效範圍 [0, 100]
    // 成功: 50
}

In [ ]:
package main

import (
    "fmt"
    "os"
)

func processFile(filename string) (content string, err error) {
    /* 使用 defer 來記錄函式執行結果 */
    defer func() {
        if err != nil {
            // 記錄錯誤並包裝
            fmt.Printf("[LOG] 處理檔案 '%s' 時發生錯誤\n", filename)
            err = fmt.Errorf("檔案處理操作失敗: %w", err)
        } else {
            fmt.Printf("[LOG] 成功處理檔案 '%s'\n", filename)
        }
    }()
    
    /* 嘗試讀取檔案 */
    data, err := os.ReadFile(filename)
    if err != nil {
        return "", err
    }
    
    /* 簡單的內容驗證 */
    if len(data) == 0 {
        return "", fmt.Errorf("檔案為空")
    }
    
    return string(data), nil
}

func main() {
    _, err := processFile("nonexistent.txt")
    if err != nil {
        fmt.Println("最終錯誤:", err)
    }
    // 輸出:
    // [LOG] 處理檔案 'nonexistent.txt' 時發生錯誤
    // 最終錯誤: 檔案處理操作失敗: open nonexistent.txt: no such file or directory
}

## panic 與 recover

`panic` 和 `recover` 是 Go 中處理嚴重錯誤的機制。`panic` 會立即停止當前函式的正常執行，並開始沿著呼叫堆疊向上展開；`recover` 可以捕獲 panic 並恢復程式的正常執行。

使用原則：
- panic 應該用於不可恢復的錯誤
- 一般錯誤應該使用 error 返回值
- recover 只能在 defer 函式中有效
- 避免過度使用 panic，它不是正常的錯誤處理機制

In [ ]:
package main

import "fmt"

func riskyOperation() {
    /* 模擬可能發生 panic 的操作 */
    slice := []int{1, 2, 3}
    fmt.Println(slice[5]) // 這行會導致 panic: index out of range
}

func safeOperation() {
    /* 使用 defer 和 recover 來捕獲 panic */
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("捕獲到 panic: %v\n", r) // 輸出: 捕獲到 panic: runtime error: index out of range [5] with length 3
            fmt.Println("程式繼續執行")
        }
    }()
    
    fmt.Println("開始執行危險操作")
    riskyOperation()
    fmt.Println("這行不會執行，因為上面發生了 panic")
}

func main() {
    fmt.Println("程式開始")
    safeOperation()
    fmt.Println("程式正常結束") // 這行會執行，因為 panic 被 recover 了
}

In [ ]:
package main

import "fmt"

func divide(a, b float64) (result float64, err error) {
    /* 使用 defer 和 recover 將 panic 轉換為 error */
    defer func() {
        if r := recover(); r != nil {
            err = fmt.Errorf("計算過程中發生嚴重錯誤: %v", r)
        }
    }()
    
    /* 故意觸發 panic 來示範 */
    if b == 0 {
        panic("除數為零，這是一個嚴重錯誤")
    }
    
    return a / b, nil
}

func main() {
    /* 正常情況 */
    result, err := divide(10, 2)
    if err != nil {
        fmt.Println("錯誤:", err)
    } else {
        fmt.Println("結果:", result) // 輸出: 結果: 5
    }
    
    /* 會觸發 panic 但被 recover 的情況 */
    result, err = divide(10, 0)
    if err != nil {
        fmt.Println("錯誤:", err) // 輸出: 錯誤: 計算過程中發生嚴重錯誤: 除數為零，這是一個嚴重錯誤
    } else {
        fmt.Println("結果:", result)
    }
}

In [ ]:
package main

import "fmt"

func processItems(items []string) {
    /* 使用 defer 來清理資源，即使發生 panic */
    defer func() {
        fmt.Println("清理資源") // 無論是否發生 panic，這行都會執行
        if r := recover(); r != nil {
            fmt.Printf("處理項目時發生 panic: %v\n", r)
            fmt.Println("執行錯誤恢復程序")
        }
    }()
    
    fmt.Println("開始處理項目")
    
    /* 故意存取超出範圍的索引 */
    for i := 0; i <= len(items); i++ { // 注意這裡是 <=，會超出範圍
        fmt.Printf("處理項目 %d: %s\n", i, items[i])
    }
    
    fmt.Println("所有項目處理完成") // 這行不會執行
}

func main() {
    items := []string{"apple", "banana", "cherry"}
    processItems(items)
    fmt.Println("主程式繼續執行") // 這行會執行
    
    // 輸出:
    // 開始處理項目
    // 處理項目 0: apple
    // 處理項目 1: banana
    // 處理項目 2: cherry
    // 清理資源
    // 處理項目時發生 panic: runtime error: index out of range [3] with length 3
    // 執行錯誤恢復程序
    // 主程式繼續執行
}

## 從 error 取得堆疊追蹤

預設情況下，Go 的 error 不包含堆疊追蹤資訊。但是可以使用第三方套件（如 `pkg/errors`）或自訂錯誤類型來添加堆疊追蹤功能。堆疊追蹤對於除錯複雜的錯誤傳播很有幫助。

獲取堆疊追蹤的方法：
- 使用 `runtime.Caller` 和 `runtime.Stack` 函式
- 使用第三方套件如 `github.com/pkg/errors`
- 在自訂錯誤類型中保存堆疊資訊

注意：堆疊追蹤會消耗額外的記憶體和 CPU，應該謹慎使用。

In [ ]:
package main

import (
    "fmt"
    "runtime"
)

/* 自訂錯誤類型，包含堆疊追蹤資訊 */
type StackError struct {
    Message string
    Stack   []uintptr
}

func (se StackError) Error() string {
    return se.Message
}

/* 取得堆疊追蹤的方法 */
func (se StackError) StackTrace() string {
    frames := runtime.CallersFrames(se.Stack)
    var result string
    
    for {
        frame, more := frames.Next()
        result += fmt.Sprintf("%s\n\t%s:%d\n", frame.Function, frame.File, frame.Line)
        if !more {
            break
        }
    }
    return result
}

/* 建立包含堆疊追蹤的錯誤 */
func NewStackError(message string) StackError {
    const depth = 32
    var pcs [depth]uintptr
    n := runtime.Callers(2, pcs[:])
    
    return StackError{
        Message: message,
        Stack:   pcs[0:n],
    }
}

func levelThree() error {
    /* 第三層函式，產生錯誤 */
    return NewStackError("第三層發生錯誤")
}

func levelTwo() error {
    /* 第二層函式，呼叫第三層 */
    return levelThree()
}

func levelOne() error {
    /* 第一層函式，呼叫第二層 */
    return levelTwo()
}

func main() {
    err := levelOne()
    if err != nil {
        fmt.Println("錯誤訊息:", err.Error())
        
        /* 如果是 StackError，顯示堆疊追蹤 */
        if stackErr, ok := err.(StackError); ok {
            fmt.Println("\n堆疊追蹤:")
            fmt.Print(stackErr.StackTrace())
        }
    }
}

In [ ]:
package main

import (
    "fmt"
    "runtime"
    "strings"
)

/* 簡單的堆疊追蹤函式 */
func getStackTrace() string {
    const size = 64 << 10 // 64KB
    buf := make([]byte, size)
    buf = buf[:runtime.Stack(buf, false)]
    return string(buf)
}

/* 記錄錯誤發生位置的函式 */
func getCallerInfo(skip int) string {
    _, file, line, ok := runtime.Caller(skip)
    if !ok {
        return "未知位置"
    }
    
    // 只顯示檔案名稱，不顯示完整路徑
    parts := strings.Split(file, "/")
    filename := parts[len(parts)-1]
    
    return fmt.Sprintf("%s:%d", filename, line)
}

func problematicFunction() error {
    /* 記錄錯誤發生的位置 */
    location := getCallerInfo(1)
    return fmt.Errorf("在 %s 發生錯誤: 模擬的業務邏輯錯誤", location)
}

func wrapperFunction() error {
    /* 包裝錯誤並添加位置資訊 */
    err := problematicFunction()
    if err != nil {
        location := getCallerInfo(1)
        return fmt.Errorf("在 %s 呼叫失敗: %w", location, err)
    }
    return nil
}

func main() {
    err := wrapperFunction()
    if err != nil {
        fmt.Println("錯誤:", err)
        
        /* 在需要詳細除錯時顯示完整堆疊追蹤 */
        fmt.Println("\n完整堆疊追蹤:")
        fmt.Print(getStackTrace())
    }
}

## 總結

Go 的錯誤處理機制基於簡單而強大的 error 介面，提供了明確且可預測的錯誤處理方式。主要概念包括：

### 關鍵重點：

1. **基本錯誤處理**: 使用 `(result, error)` 模式，明確檢查每個錯誤
2. **簡單錯誤建立**: 使用 `errors.New()` 和 `fmt.Errorf()` 建立錯誤訊息
3. **哨兵錯誤**: 預定義的錯誤值，便於特定錯誤的識別和處理
4. **自訂錯誤類型**: 實作 `Error()` 方法來提供豐富的錯誤資訊
5. **錯誤包裝**: 使用 `fmt.Errorf` 與 `%w` 動詞建立錯誤鏈
6. **錯誤檢查**: 使用 `errors.Is` 和 `errors.As` 處理包裝錯誤
7. **defer 錯誤處理**: 在函式返回前統一處理和包裝錯誤
8. **panic/recover**: 處理嚴重錯誤，但應該謹慎使用
9. **堆疊追蹤**: 提供詳細的錯誤發生位置資訊，便於除錯

### 最佳實踐：

- 總是檢查和處理錯誤，不要忽略
- 為錯誤提供足夠的上下文資訊
- 使用錯誤包裝來建立清晰的錯誤鏈
- 優先使用 error 返回值而不是 panic
- 在適當的層級處理錯誤，不要過度傳播
- 使用有意義的錯誤訊息和類型